<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #12: Análisis de sentimiento
`Fecha de entrega: Mayo 27, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

In [6]:
import pandas as pd
import tweepy

from flair.models import TextClassifier
from flair.data import Sentence

keys = pd.read_csv("C:/Users/andre/OneDrive/Documentos/Especialización/NLP/tw.csv", header=None, delimiter=";")
keys = dict(zip(keys[0],keys[1]))
keys.keys()
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)


In [4]:
tweets = tweepy.Cursor(api.search,
                       q=['protesta'],
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(150)

In [5]:
t = [tweet for tweet in tweets]


t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']

import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions
0,RT @SergioComunes: Esta foto de César Melgarej...,2021-05-22 22:28:54,Bogotá,Facatativá,1396231623570505728,https://twitter.com/i/web/stuatus/139623162357...,"RT Esta foto de César Melgarejo, muestra por ...",[],[],[@SergioComunes:]
1,RT @SARGENTOCHALA: Así se reprime la protesta ...,2021-05-22 22:28:54,Bogotá,Bogotá,1396231622224228352,https://twitter.com/i/web/stuatus/139623162222...,RT Así se reprime la protesta social en COLOM...,[],[],[@SARGENTOCHALA:]
2,RT @MeDicenWally: Las protestas han sido de la...,2021-05-22 22:28:50,Bogotá,"Gold Coast, Queensland",1396231605505699845,https://twitter.com/i/web/stuatus/139623160550...,RT Las protestas han sido de la ciudadanía. E...,[],[],[@MeDicenWally:]
3,RT @SergioComunes: Esta foto de César Melgarej...,2021-05-22 22:28:40,Bogotá,New jersey,1396231565739544576,https://twitter.com/i/web/stuatus/139623156573...,"RT Esta foto de César Melgarejo, muestra por ...",[],[],[@SergioComunes:]
4,RT @TembloresOng: Sancionar el uso de armas de...,2021-05-22 22:28:39,Bogotá,"Bogotá, D.C., Colombia",1396231561322721282,https://twitter.com/i/web/stuatus/139623156132...,RT Sancionar el uso de armas de fuego (prohib...,[],[],[@TembloresOng:]
...,...,...,...,...,...,...,...,...,...,...
145,RT @SARGENTOCHALA: Así se reprime la protesta ...,2021-05-22 22:17:45,Bogotá,"Bucaramanga, Colombia",1396228818789142528,https://twitter.com/i/web/stuatus/139622881878...,RT Así se reprime la protesta social en COLOM...,[],[],[@SARGENTOCHALA:]
146,RT @TembloresOng: Reafirmar los derechos const...,2021-05-22 22:17:40,Bogotá,,1396228798559838208,https://twitter.com/i/web/stuatus/139622879855...,RT Reafirmar los derechos constitucionales a ...,[],[],[@TembloresOng:]
147,RT @SARGENTOCHALA: Así se reprime la protesta ...,2021-05-22 22:17:30,Bogotá,,1396228753307602945,https://twitter.com/i/web/stuatus/139622875330...,RT Así se reprime la protesta social en COLOM...,[],[],[@SARGENTOCHALA:]
148,@Angugo71 @nogogu @IvanDuque Los bloqueos u ob...,2021-05-22 22:17:29,Bogotá,"Santiago de Compostela, España",1396228751206330372,https://twitter.com/i/web/stuatus/139622875120...,Los bloqueos u obstrucción a la vía pública...,[https://t.co/2iBLkKbI6e],[],"[@Angugo71, @nogogu, @IvanDuque]"


In [7]:
classifier = TextClassifier.load('sentiment')

#función para aplicar en ingles y tener mismo rango que las otras dos. 
def sent_flair(texto):
    frase = Sentence(texto)
    classifier.predict(frase)
    result = frase.labels[0]
    label = result.value
    score = result.score
    if label == 'NEGATIVE':
        return score * -1
    return score

data['sent_flair'] = data['texto_limpio'].apply(lambda val: sent_flair(val))
data.head()

2021-05-22 17:35:47,981 loading file C:\Users\andre\.flair\models\sentiment-en-mix-distillbert_4.pt


,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions,sent_flair
0,RT @SergioComunes: Esta foto de César Melgarej...,2021-05-22 22:28:54,Bogotá,Facatativá,1396231623570505728,https://twitter.com/i/web/stuatus/139623162357...,"RT Esta foto de César Melgarejo, muestra por ...",[],[],[@SergioComunes:],0.992589
1,RT @SARGENTOCHALA: Así se reprime la protesta ...,2021-05-22 22:28:54,Bogotá,Bogotá,1396231622224228352,https://twitter.com/i/web/stuatus/139623162222...,RT Así se reprime la protesta social en COLOM...,[],[],[@SARGENTOCHALA:],-0.994930
2,RT @MeDicenWally: Las protestas han sido de la...,2021-05-22 22:28:50,Bogotá,"Gold Coast, Queensland",1396231605505699845,https://twitter.com/i/web/stuatus/139623160550...,RT Las protestas han sido de la ciudadanía. E...,[],[],[@MeDicenWally:],0.750408
3,RT @SergioComunes: Esta foto de César Melgarej...,2021-05-22 22:28:40,Bogotá,New jersey,1396231565739544576,https://twitter.com/i/web/stuatus/139623156573...,"RT Esta foto de César Melgarejo, muestra por ...",[],[],[@SergioComunes:],0.992589
4,RT @TembloresOng: Sancionar el uso de armas de...,2021-05-22 22:28:39,Bogotá,"Bogotá, D.C., Colombia",1396231561322721282,https://twitter.com/i/web/stuatus/139623156132...,RT Sancionar el uso de armas de fuego (prohib...,[],[],[@TembloresOng:],0.979981


###  `[25 pts]` Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [38]:
positivo= data[data.sent_flair==data.sent_flair.max()]
print("el tuit más positivo es",positivo.texto_limpio.head(1).values, "con un valor de", positivo.sent_flair.head(1).values)


el tuit más positivo es ['RT  Todos los que puedan lleven cascos, guantes, gafas y máscara para el portal de las américas.\nLa orden por parte de la a…'] con un valor de [0.99402595]


In [39]:
negativo= data[data.sent_flair==data.sent_flair.min()]
print("el tuit más negativo es",negativo.texto_limpio.head(1).values, "con un valor de", negativo.sent_flair.head(1).values)


el tuit más negativo es ['Los bloqueos u obstrucción a la vía pública no constituyen una expresión de la protesta pacífica, pues violentan derechos y producen daños irreparables a la población '] con un valor de [-0.99946052]
